<a href="https://colab.research.google.com/github/milieureka/derivative-pricing/blob/main/option_pricing_Heston_and_Merton_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libraries

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
np.random.seed(42) #Set seed for consistent results throughout code

# Parameter

In [ ]:
S0=80 #Intial stock price
r=5.5/100 #Risk-free rate of 5.5%
sigma=35/100 #35% volatility of volatility
T=3/12 #3 months to Maturity
K=S0 #ATM European call and put
v0=3.2/100 #initial volatility of 3.2%
kappa_v=1.85
theta_v=0.045
M=90 #Number of paths to simulate in Monte-Carlo
Ite=1000000 #Number of steps in Monte-Carlo
t=0 #Price options at time 0

# Step 1

### Pricing ATM European Call and Put options under Heston model with -0.30 correlation

In [ ]:
def random_number_generator(M,Ite):
    """Generate random numbers fron standard normal distribution
    given number of steps and number of iterations
    for Monte-Carlo simulation
    INPUTS: M - number of steps in MC, Ite - number of iterations in MC
    OUTPUT: rand - array of random numbers"""
    rand=np.random.standard_normal((2,M+1,Ite))
    return rand

In [ ]:
def cholesky_decomp(rho):
    """Perform Cholesky decomposition of covariance matrix for a given
    correlation between Weiner processes
    INPUT: rho - correlation
    OUTPUT: cholesky_matrix - lower Cholesky decomposition of covariance
    matrix to account for correlation between Weiner processes"""
    cov_matrix=np.zeros((2,2),dtype=np.float64)
    cov_matrix[0]=[1.0,rho]
    cov_matrix[1]=[rho,1.0]
    cholesky_matrix=np.linalg.cholesky(cov_matrix)
    return cholesky_matrix

In [ ]:
def Heston_vol(v0,kappa,theta,sigma,T,M,Ite,rand,row,cho_matrix):
    """Calculate stochastic volatility under the Heston model
    INPUTS: v0 - initial volatility, kappa - rate of mean-reversion,
    theta - long run average variance, sigma - volatility of volatility,
    T - time to maturity, M - number of steps in MC, Ite - number of iterations in MC,
    rand - generated random numbers from standard normal,
    row - row of array to apply random numbers,
    cho_matrix - lower Cholesky decomposition of covariance matrix
    OUTPUT: simulated volatility under Heston model"""
    dt=T/M #Time increment
    ran=np.tensordot(cho_matrix, rand,axes=([1],[0])) #Apply Cholesky decomposition
    ran_row=ran[row,1:] #Initialise random numbers
    v=np.zeros((M+1,Ite),dtype=np.float64) #Create volatility array
    v[0]=v0 #Set initial volatility
    #Heston volatility SDE
    v_increment=kappa*theta*dt-kappa*v[:-1]*dt+sigma*np.sqrt(dt)*ran_row*np.sqrt(np.maximum(v[:-1],0))
    #Volaility cannnot be negative
    v[1:]=np.maximum(v[0]+np.cumsum(v_increment,axis=0),0)
    return v

In [ ]:
def Heston_paths(S0,r,v,T,M,Ite,rand,row,cho_matrix):
    """Simulate stock paths under the Heston model using Monte-Carlo methods
    INPUTS: S0 - intial stock (underlying) price, r - risk-free rate,
    v - stochastic volatility under Heston model, T - time to maturity,
    M - number of steps in MC, Ite - number of iterations in MC,
    rand - generated random numbers from standard normal,
    row - row of array to apply random numbers,
    cho_matrix - Lower Cholesky decomposition of covariance matrix
    OUTPUT: S - simulated stock price paths under Heston model"""
    dt=T/M
    ran=np.tensordot(cho_matrix, rand,axes=([1],[0]))
    ran_row=ran[row,1:]
    S=np.zeros((M+1,Ite),dtype=np.float64)
    S[0]=S0
    S[1:]=S0*np.cumprod(np.exp((r-0.5*v[1:])*dt+np.sqrt(v[1:])*ran_row*np.sqrt(dt)),axis=0)
    return S

In [ ]:
def European_price(S,K,r,T,t,option_type='call'):
    """Calculate vanilla European Call and Put option prices using Monte-Carlo simulation
    INPUTS: S - stock price paths, K - strike price, r - risk-free rate,
    T - time to maturity, t - time to calculate option prices at,
    option_type - either 'call' or 'put'
    OUTPUT: Price of vanilla European call or put option"""
    if option_type.lower()=='call':
        payoff=np.maximum(S[-1:]-K,0)
    elif option_type.lower()=='put':
        payoff=np.maximum(K-S[-1,:],0)
    average=np.mean(payoff)
    return np.exp(-r*(T-t))*average

In [ ]:
rho_1=-0.30 #Correlation of -0.30
#Perform Cholesky decompposition
cho_matrix1=cholesky_decomp(rho_1)
rand=random_number_generator(M, Ite) #Generate random number
row=1  #Apply random numbers to first row of simulated volatility and stock paths
#Simulate Heston volatility
v_1=Heston_vol(v0,kappa_v,theta_v,sigma,T,M,Ite,rand,row,cho_matrix1)
#Simulate Heston stock paths
S_heston1=Heston_paths(S0,r,v_1,T,M,Ite,rand,row,cho_matrix1)
#Price European call option under Heston model
european_call_heston1=np.round(European_price(S_heston1,K,r,T,t,option_type='call'),2)
print(f"European call option price under Heston model with -0.30 correlation is: ${european_call_heston1}")
#Price European put option under Heston model
european_put_heston1=np.round(European_price(S_heston1,K,r,T,t,option_type='put'),2)
print(f"European put option price under Heston model with -0.30 correlation is: ${european_put_heston1}")

European call option price under Heston model with -0.30 correlation is: $3.86
European put option price under Heston model with -0.30 correlation is: $2.72


### Pricing ATM European Call and Put options under Heston model with -0.70 correlation

In [ ]:
rho_2=-0.70 #Correlation of -0.70

In [ ]:
#Redo Cholesky decomposition
cho_matrix2=cholesky_decomp(rho_2)
#Regenerate random number
rand=random_number_generator(M, Ite)
#Re-simulate Heston volatility and stock paths
v_2=Heston_vol(v0,kappa_v,theta_v,sigma,T,M,Ite,rand,row,cho_matrix2)
S_heston2=Heston_paths(S0,r,v_2,T,M,Ite,rand,row,cho_matrix2)
european_call_heston2=np.round(European_price(S_heston2,K,r,T,t,option_type='call'),2)
print(f"European call option price under Heston model with -0.70 correlation is: ${european_call_heston2}")
#Price European put option under Heston model
european_put_heston2=np.round(European_price(S_heston2,K,r,T,t,option_type='put'),2)
print(f"European put option price under Heston model with -0.70 correlation is: ${european_put_heston2}")

European call option price under Heston model with -0.70 correlation is: $3.86
European put option price under Heston model with -0.70 correlation is: $2.72


## Check Put-Call Parity For number 5 and 6

In [ ]:
def check_put_call_parity(C, P, S0, K, r, T):
    """Check put-call parity for European options.
    INPUTS:
    C - Call option price
    P - Put option price
    S0 - Initial stock price
    K - Strike price
    r - Risk-free rate
    T - Time to maturity
    OUTPUT:
    Parity result and discrepancy"""
    lhs = C - P
    rhs = S0 - K * np.exp(-r * T)
    discrepancy = np.abs(lhs - rhs)
    return lhs, rhs, discrepancy

# Calculate put-call parity for correlation -0.30
lhs_1, rhs_1, discrepancy_1 = check_put_call_parity(
    european_call_heston1, european_put_heston1, S0, K, r, T
)

print("\nPut-Call Parity Check for Correlation -0.30:")
print(f"LHS (C - P): {lhs_1:.2f}")
print(f"RHS (S0 - K * exp(-r * T)): {rhs_1:.2f}")
print(f"Discrepancy: {discrepancy_1:.2e}")

# Calculate put-call parity for correlation -0.70
lhs_2, rhs_2, discrepancy_2 = check_put_call_parity(
    european_call_heston2, european_put_heston2, S0, K, r, T
)

print("\nPut-Call Parity Check for Correlation -0.70:")
print(f"LHS (C - P): {lhs_2:.2f}")
print(f"RHS (S0 - K * exp(-r * T)): {rhs_2:.2f}")
print(f"Discrepancy: {discrepancy_2:.2f}")



Put-Call Parity Check for Correlation -0.30:
LHS (C - P): 1.14
RHS (S0 - K * exp(-r * T)): 1.09
Discrepancy: 4.75e-02

Put-Call Parity Check for Correlation -0.70:
LHS (C - P): 1.14
RHS (S0 - K * exp(-r * T)): 1.09
Discrepancy: 0.05


**Calculate delta and gamma for each of the options in Q5 and Q6**

In [ ]:
def Heston_european_delta(S0,S_increment,r,K,v0,kappa,theta,sigma,T,t,M,Ite,rand,row,cho_matrix,option_type='call'):
    """Calculate delta for vanilla European call and put options under Heston model
    INPUTS: S0 - Initial stock price, S_increment - amount to change initial
    stock price by to numerically calculate delta,
    r - risk-free rate, K - strike price, v0 - initiality volatility,
    kappa - rate of mean-reversion, theta - long run average variance,
    sigma - volatility of volatility, T - time to maturity,
    t - time to calculate option prices at, M - number of steps in MC,
    Ite - number of iterations in MC, rand - generated random numbers,
    row - row of array to apply random numbers,
    cho_matrix - Lower Cholesky decomposition of covariance matrix,
    option_type - either 'call' or 'put'
    OUTPUT: Delta for vanilla European option under Heston model calculated
    using first-order foward finite-difference"""
    v=Heston_vol(v0,kappa,theta,sigma,T,M,Ite,rand,row,cho_matrix)
    S1=Heston_paths(S0,r,v,T,M,Ite,rand,row,cho_matrix)
    S2=Heston_paths(S0+S_increment,r,v,T,M,Ite,rand,row,cho_matrix)
    option_noinc=European_price(S1,K,r,T,t,option_type)
    option_inc=European_price(S2,K,r,T,t,option_type)
    return (option_inc-option_noinc)/S_increment

In [ ]:
def Heston_european_gamma(S0,S_increment,r,K,v0,kappa,theta,sigma,T,t,M,Ite,rand,row,cho_matrix,option_type='call'):
    """Calculate gamma for vanilla European call and put options under Heston model
    INPUTS: S0 - Initial stock price, S_increment - amount to change initial
    stock price by to numerically calculate delta,
    r - risk-free rate, K - strike price, v0 - initiality volatility,
    kappa - rate of mean-reversion, theta - long run average variance,
    sigma - volatility of volatility, T - time to maturity,
    t - time to calculate option prices at, M - number of steps in MC,
    Ite - number of iterations in MC, rand - generated random numbers,
    row - row of array to apply random numbers,
    cho_matrix - Lower Cholesky decomposition of covariance matrix,
    option_type - either 'call' or 'put'
    OUTPUT: Gamma for vanilla European option under Heston model calculated
    using second-order central finite difference"""
    v=Heston_vol(v0,kappa,theta,sigma,T,M,Ite,rand,row,cho_matrix)
    S1=Heston_paths(S0,r,v,T,M,Ite,rand,row,cho_matrix)
    S2=Heston_paths(S0+S_increment,r,v,T,M,Ite,rand,row,cho_matrix)
    S3=Heston_paths(S0-S_increment,r,v,T,M,Ite,rand,row,cho_matrix)
    option_noinc=European_price(S1,K,r,T,t,option_type)
    option_plus=European_price(S2,K,r,T,t,option_type)
    option_minus=European_price(S3,K,r,T,t,option_type)
    return (option_plus-2*option_noinc+option_minus)/(S_increment**2)

Delta and gamma for Call and Put options with rho=-0.30 (Q5)

In [ ]:
S_increment=0.0001*S0 #Small change in initial stock price of 0.01%
#Delta and gamma for Call and Put options with rho=-0.30
delta_call1=Heston_european_delta(S0, S_increment, r, K, v0, kappa_v, theta_v, sigma, T, t, M, Ite, rand, row, cho_matrix1,option_type='call')
delta_put1=Heston_european_delta(S0, S_increment, r, K, v0, kappa_v, theta_v, sigma, T, t, M, Ite, rand, row, cho_matrix1,option_type='put')
#Gamma is the same for both Call and Put Options, so only need to evaluate for Call option
gamma1=Heston_european_gamma(S0,S_increment,r,K,v0,kappa_v,theta_v,sigma,T,t,M,Ite,rand,row,cho_matrix1,option_type='call')
print(f"Delta for European Call option under Heston model with -0.30 correlation is: {delta_call1}")
print(f"Delta for European Put option under Heston model with -0.30 correlation is: {delta_put1}")
print(f"Gamma for European options under Heston model with -0.30 correlation is: {gamma1}")

Delta for European Call option under Heston model with -0.30 correlation is: 0.574372831690817
Delta for European Put option under Heston model with -0.30 correlation is: -0.4261514819765555
Gamma for European options under Heston model with -0.30 correlation is: 0.04782608146319811


Delta and gamma for Call and Put options with rho=-0.70 (Q6)

In [ ]:
delta_call2=Heston_european_delta(S0, S_increment, r, K, v0, kappa_v, theta_v, sigma, T, t, M, Ite, rand, row, cho_matrix2,option_type='call')
delta_put2=Heston_european_delta(S0, S_increment, r, K, v0, kappa_v, theta_v, sigma, T, t, M, Ite, rand, row, cho_matrix2,option_type='put')
#Gamma is the same for both Call and Put Options, so only need to evaluate for Call option
gamma2=Heston_european_gamma(S0,S_increment,r,K,v0,kappa_v,theta_v,sigma,T,t,M,Ite,rand,row,cho_matrix2,option_type='call')
print(f"Delta for European Call option under Heston model with -0.70 correlation is: {delta_call2}")
print(f"Delta for European Put option under Heston model with -0.70 correlation is: {delta_put2}")
print(f"Gamma for European options under Heston model with -0.70 correlation is: {gamma2}")

Delta for European Call option under Heston model with -0.70 correlation is: 0.5752997434682605
Delta for European Put option under Heston model with -0.70 correlation is: -0.425263354597083
Gamma for European options under Heston model with -0.70 correlation is: 0.05103275705492516


## Merton Model

- Using the Merton Model, price an ATM European call and put with jump intensity parameter equal to 0.75.
- Using the Merton Model, price an ATM European call and put with jump intensity parameter equal to 0.25.
- Calculate delta and gamma for each of the options in 2 requirement above

In [ ]:
# Parameters
S0 = 80  # Initial stock price
r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3 / 12  # Time to maturity (3 months)
mu = -0.5  # Jump mean
delta = 0.22  # Jump standard deviation
lambdas = [0.75, 0.25]  # Jump intensities
K = S0  # ATM strike price

# Simulation parameters
Ite = 3000000  # Number of iterations
M = 50  # Number of steps
dt = T / M  # Time step

# Function to calculate European option price and Greeks
def merton_option_pricing(S0, r, sigma, T, mu, delta, lamb, K, Ite, M):
    dt = T / M
    rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)  # Adjustment for jumps

    # Pre-generate random numbers
    z1 = np.random.standard_normal((M, Ite))  # For diffusion
    z2 = np.random.standard_normal((M, Ite))  # For jump size
    y = np.random.poisson(lamb * dt, (M, Ite))  # For jump frequency

    # Initialize stock price matrix
    SM = np.zeros((M + 1, Ite))
    SM[0] = S0

    # Simulate paths
    for t in range(1, M + 1):
        jump_component = (np.exp(mu + delta * z2[t - 1]) - 1) * y[t - 1]
        diffusion_component = (r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t - 1]
        SM[t] = SM[t - 1] * (np.exp(diffusion_component) + jump_component)
        SM[t] = np.maximum(SM[t], 0.00001)  # Ensure non-negative prices

    # Option payoffs
    ST = SM[-1]
    call_payoff = np.maximum(ST - K, 0)
    put_payoff = np.maximum(K - ST, 0)

    # Discounted prices
    call_price = np.exp(-r * T) * np.mean(call_payoff)
    put_price = np.exp(-r * T) * np.mean(put_payoff)

    # Greeks (finite difference method)
    epsilon = 1e-4
    S_up = S0 * (1 + epsilon)
    S_down = S0 * (1 - epsilon)

    # Re-run simulations for delta and gamma calculations
    SM_up = np.zeros((M + 1, Ite))
    SM_down = np.zeros((M + 1, Ite))
    SM_up[0] = S_up
    SM_down[0] = S_down

    for t in range(1, M + 1):
        jump_up = (np.exp(mu + delta * z2[t - 1]) - 1) * y[t - 1]
        jump_down = jump_up

        diffusion_up = (r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t - 1]
        diffusion_down = diffusion_up

        SM_up[t] = SM_up[t - 1] * (np.exp(diffusion_up) + jump_up)
        SM_down[t] = SM_down[t - 1] * (np.exp(diffusion_down) + jump_down)

        SM_up[t] = np.maximum(SM_up[t], 0.00001)
        SM_down[t] = np.maximum(SM_down[t], 0.00001)

    ST_up = SM_up[-1]
    ST_down = SM_down[-1]

    call_delta = (np.mean(np.maximum(ST_up - K, 0)) - np.mean(np.maximum(ST_down - K, 0))) / (2 * S0 * epsilon)
    put_delta = (np.mean(np.maximum(K - ST_down, 0)) - np.mean(np.maximum(K - ST_up, 0))) / (2 * S0 * epsilon)

    call_gamma = (
        (np.mean(np.maximum(ST_up - K, 0)) + np.mean(np.maximum(ST_down - K, 0)) - 2 * np.mean(call_payoff))
        / (S0**2 * epsilon**2)
    )
    put_gamma = (
        (np.mean(np.maximum(K - ST_down, 0)) + np.mean(np.maximum(K - ST_up, 0)) - 2 * np.mean(put_payoff))
        / (S0**2 * epsilon**2)
    )

    return call_price, put_price, call_delta, put_delta, call_gamma, put_gamma

## Check Put Parity

In [ ]:
# Results
for lamb in lambdas:
    call_price, put_price, call_delta, put_delta, call_gamma, put_gamma = merton_option_pricing(
        S0, r, sigma, T, mu, delta, lamb, K, Ite, M
    )

    # Put-Call Parity Check
    parity_difference = call_price - put_price - (S0 - K * np.exp(-r * T))
    assert abs(parity_difference) <= 0.05, f"Put-Call Parity violated: {parity_difference}"

    print(f"Jump intensity (lambda): {lamb}")
    print(f"Call Price: {call_price:.4f}, Put Price: {put_price:.4f}")
    print(f"Call Delta: {call_delta:.4f}, Put Delta: {put_delta:.4f}")
    print(f"Call Gamma: {call_gamma:.6f}, Put Gamma: {put_gamma:.6f}")
    print(f"Put-Call Parity Check Passed (Difference: {parity_difference:.8f})")
    print("-")


Jump intensity (lambda): 0.75
Call Price: 8.3084, Put Price: 7.2174
Call Delta: 0.6573, Put Delta: 0.3565
Call Gamma: 0.021904, Put Gamma: 0.021904
Put-Call Parity Check Passed (Difference: -0.00152982)
-
Jump intensity (lambda): 0.25
Call Price: 6.8237, Put Price: 5.7320
Call Delta: 0.6059, Put Delta: 0.4079
Call Gamma: 0.026979, Put Gamma: 0.026979
Put-Call Parity Check Passed (Difference: -0.00079382)
-


In [ ]:
S0=80 #Intial stock price
r=5.5/100 #Risk-free rate of 5.5%
sigma=35/100 #35% volatility of volatility
T=3/12 #3 months to Maturity
K=S0 #ATM European call and put
v0=3.2/100 #initial volatility of 3.2%
kappa_v=1.85
theta_v=0.045
M=90 #Number of paths to simulate in Monte-Carlo
Ite=1000000 #Number of steps in Monte-Carlo
t=0 #Price options at time 0

In [ ]:
# Moneyness values and corresponding strike prices
moneyness_values = [0.85, 0.90, 0.95, 1.00, 1.05, 1.10, 1.15]
strike_prices = [S0 / m for m in moneyness_values]

# Function to calculate prices for different strikes
def price_for_strikes_heston_merton(S0, r, sigma, T, v0, kappa_v, theta_v, K_list, M, Ite, mu, delta, lambdas, rho):
    heston_results = []
    merton_results = []

    for K in K_list:
        # Heston Model
        cho_matrix = cholesky_decomp(rho)
        rand = random_number_generator(M, Ite)
        row = 1
        v = Heston_vol(v0, kappa_v, theta_v, sigma, T, M, Ite, rand, row, cho_matrix)
        S_heston = Heston_paths(S0, r, v, T, M, Ite, rand, row, cho_matrix)
        call_price_heston = European_price(S_heston, K, r, T, t, option_type='call')
        put_price_heston = European_price(S_heston, K, r, T, t, option_type='put')
        heston_results.append((K, call_price_heston, put_price_heston))

        # Merton Model
        merton_prices = []
        for lamb in lambdas:
            call_price, put_price, _, _, _, _ = merton_option_pricing(
                S0, r, sigma, T, mu, delta, lamb, K, Ite, M
            )
            merton_prices.append((call_price, put_price))
        merton_results.append((K, merton_prices))

    return heston_results, merton_results

# Run the models for the 7 strike prices
rho = -0.30  # Using -0.30 correlation for Heston model
heston_prices, merton_prices = price_for_strikes_heston_merton(
    S0, r, sigma, T, v0, kappa_v, theta_v, strike_prices, M, Ite, mu, delta, lambdas, rho
)

# Display results
print("Results for Different Strike Prices (Moneyness: 0.85 to 1.15):\n")
for i, K in enumerate(strike_prices):
    print(f"Strike Price: {K:.2f} (Moneyness: {moneyness_values[i]})")
    print(f"Heston Call Price: {heston_prices[i][1]:.4f}, Heston Put Price: {heston_prices[i][2]:.4f}")
    print("Merton Model:")
    for j, lamb in enumerate(lambdas):
        print(f"  Lambda {lamb:.2f}: Call Price: {merton_prices[i][1][j][0]:.4f}, Put Price: {merton_prices[i][1][j][1]:.4f}")
    print("-" * 50)


Results for Different Strike Prices (Moneyness: 0.85 to 1.15):

Strike Price: 94.12 (Moneyness: 0.85)
Heston Call Price: 0.3050, Heston Put Price: 13.1047
Merton Model:
  Lambda 0.75: Call Price: 2.8130, Put Price: 15.6457
  Lambda 0.25: Call Price: 2.0034, Put Price: 14.8258
--------------------------------------------------
Strike Price: 88.89 (Moneyness: 0.9)
Heston Call Price: 0.8944, Heston Put Price: 8.5388
Merton Model:
  Lambda 0.75: Call Price: 4.3700, Put Price: 11.9735
  Lambda 0.25: Call Price: 3.2743, Put Price: 10.9383
--------------------------------------------------
Strike Price: 84.21 (Moneyness: 0.95)
Heston Call Price: 2.0517, Heston Put Price: 5.0753
Merton Model:
  Lambda 0.75: Call Price: 6.2126, Put Price: 9.2671
  Lambda 0.25: Call Price: 4.8950, Put Price: 7.9571
--------------------------------------------------
Strike Price: 80.00 (Moneyness: 1.0)
Heston Call Price: 3.8635, Heston Put Price: 2.7298
Merton Model:
  Lambda 0.75: Call Price: 8.3086, Put Price: 

#Step 2

### Q.13 - Pricing American call options under Heston and Merton models

In [ ]:
def American_call_price(S,K,r,T,t,M):
    """Calculate price of American call option (accounting for early exercise)
    using Monte-Carlo simulation
    INPUTS: S - simulated stock paths, K - strike price, r - risk-free rate,
    T - time to maturity, t - time to calculate options prices at,
    M - number of steps in MC simulation
    OUTPUT: call_price - price of American call option"""
    dt=T/M
    payoff=np.maximum(S[-1,:]-K,0)
    for i in range(M-2,-1,-1):
        in_the_money=S[i,:]>K
        if np.any(in_the_money):
            future_payoff=np.exp(-r*dt)*payoff
            X=S[i,:][in_the_money].reshape(-1,1)
            Y=future_payoff[in_the_money]
            reg=LinearRegression()
            reg.fit(X,Y)
            continuation_value=reg.predict(S[i,:].reshape(-1,1))
            exercise_value=np.maximum(S[i,:]-K,0)
            payoff=np.maximum(exercise_value,continuation_value)
    call_price=np.exp(-r*(T-t))*np.mean(payoff)
    return call_price

In [ ]:
def Merton_paths(S0,r,sigma,lamb,mu,delta,T,M,Ite):
    """Simulate stock paths under the Merton (1976) model
    INPUTS: S0 - initial stock price, r - risk-free rate,
    lamb - intensity of Poisson jump process,
    mu - average jump size, delta - standard deviation of jump process,
    T - time to maturity, M - number of steps in MC,
    Ite - number of iterations in MC
    OUTPUT: S - simulated stock paths under the Merton model"""
    dt=T/M
    #Correction to drift term to maintain risk-neutral measure
    rj=lamb*(np.exp(mu+0.5*delta**2)-1)
    np.random.seed(0)
    z1=np.random.standard_normal((M,Ite))
    z2=np.random.standard_normal((M,Ite))
    y=np.random.poisson(lamb*dt,(M,Ite))
    S=np.zeros((M+1,Ite))
    S[0]=S0
    S[1:]=S0*np.cumprod(np.exp((r-rj-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z1)
                        +y*(np.exp(mu+delta*z2)-1),
                        axis=0)
    S=np.maximum(S,0.00001)
    return S
#Amercian call option price under Heston model with rho=-0.30
american_call_heston=np.round(American_call_price(S_heston1,K,r,T,t,M),2)
print(f"American call option price under Heston model with -0.30 correlation is: ${american_call_heston}")

American call option price under Heston model with -0.30 correlation is: $11.36


In [ ]:
#Parameters for Merton model
mu=-0.5
delta=0.22
lamb=0.75
#Simulate stock paths under Merton model
S_merton=Merton_paths(S0,r,sigma,lamb,mu,delta,T,M,Ite)
#Amercian call option price under Merton model
american_call_merton=np.round(American_call_price(S_merton,K,r,T,t,M),2)
print(f"American call option price under Merton model with jump intensity 0.75 is: ${american_call_merton}")
#For comparison, calculate price of vanilla European call option under Merton model
european_call_merton=np.round(European_price(S_merton,K,r,T,t,option_type='call'),2)
print(f"European call option price under Merton model with jump intensity 0.75 is: ${european_call_merton}")

American call option price under Merton model with jump intensity 0.75 is: $14.4
European call option price under Merton model with jump intensity 0.75 is: $15.12


Q13. Repeat Questions 8 for the case of an American call option, using Merton Jump Diffusion.

In [ ]:
import numpy as np

# Parameters
S0 = 80  # Initial stock price
K = 80   # Strike price (ATM)
r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility of diffusion component
T = 0.25  # Time to maturity in years (3 months)
lambda_jump = 0.75  # Jump intensity
mu_jump = -0.5  # Mean jump size (logarithmic)
delta_jump = 0.22  # Jump size standard deviation

# Monte Carlo parameters
N_sim = 700000  # Number of simulations
N_steps = 50  # Time steps
dt = T / N_steps  # Time step size

# Derived parameters
r_j = lambda_jump * (np.exp(mu_jump + 0.5 * delta_jump**2) - 1)  # Drift correction

# Seed for reproducibility
np.random.seed(42)

# Pre-compute constants for efficiency
drift = (r - r_j - 0.5 * sigma**2) * dt
diffusion = sigma * np.sqrt(dt)

# Simulate stock paths
S_paths = np.zeros((N_sim, N_steps + 1))
S_paths[:, 0] = S0

for t in range(1, N_steps + 1):
    z1 = np.random.normal(0, 1, N_sim)  # Standard normal for diffusion
    y = np.random.poisson(lambda_jump * dt, N_sim)  # Poisson random variable for jumps
    z2 = np.random.normal(0, 1, N_sim)  # Standard normal for jump sizes

    jump_component = np.exp(mu_jump + delta_jump * z2) - 1
    S_paths[:, t] = S_paths[:, t - 1] * np.exp(drift + diffusion * z1) * (1 + jump_component)**y

# Payoff at maturity
call_payoff = np.maximum(S_paths[:, -1] - K, 0)
put_payoff = np.maximum(K - S_paths[:, -1], 0)

# Discounted payoffs for European options (baseline)
C_European = np.exp(-r * T) * np.mean(call_payoff)
P_European = np.exp(-r * T) * np.mean(put_payoff)

# American options using Least Squares Monte Carlo (LSM)
call_option_values = np.copy(call_payoff)
put_option_values = np.copy(put_payoff)

# Work backward through time steps for early exercise
for t in range(N_steps - 1, 0, -1):
    # In-the-money paths
    call_ITM = S_paths[:, t] > K
    put_ITM = S_paths[:, t] < K

    # Regression for continuation value
    if np.any(call_ITM):
        X_call = S_paths[call_ITM, t]
        Y_call = np.exp(-r * dt) * call_option_values[call_ITM]
        continuation_call = np.polyfit(X_call, Y_call, 2)
        expected_call = np.polyval(continuation_call, X_call)
        call_option_values[call_ITM] = np.maximum(call_payoff[call_ITM], expected_call)

    if np.any(put_ITM):
        X_put = S_paths[put_ITM, t]
        Y_put = np.exp(-r * dt) * put_option_values[put_ITM]
        continuation_put = np.polyfit(X_put, Y_put, 2)
        expected_put = np.polyval(continuation_put, X_put)
        put_option_values[put_ITM] = np.maximum(put_payoff[put_ITM], expected_put)

# Discounted American option values
C_American = np.exp(-r * dt) * np.mean(call_option_values)
P_American = np.exp(-r * dt) * np.mean(put_option_values)
print ("American Call option price is:",C_American)


American Call option price is: 27.26589884598237


Parity check

In [ ]:
# Parameters
discounted_strike = K * np.exp(-r * T)  # Discounted strike price

# Check put-call parity
parity_difference = (C_American - P_American) - (S0 - discounted_strike)

# Output results
print("Put-Call Parity Difference:", parity_difference)
if np.isclose(parity_difference, 0, atol=0.01):
    print("Put-call parity holds within the tolerance.")
else:
    print("Put-call parity does not hold.")

Put-Call Parity Difference: -0.001326668287337185
Put-call parity holds within the tolerance.


### Q14 - Pricing European up-and-in-call option under Heston model with -0.70 correlation

In [ ]:
def uai_barrier_european_call(S,K,r,T,t,B):
    """Calculate price of up-and-in European call option using MC methods
    INPUTS: S - simulated stock paths, K - strike price,
    r - risk-free rate, T - time to maturity, t - time to calculate option price at,
    B - Barrier level for the option
    OUTPUT: Price of up-and-in barrier option"""
    barrier_breached=(S>=B).any(axis=0)
    payoff=np.where(barrier_breached,np.maximum(S[-1,:]-K,0),0)
    average=np.mean(payoff)
    return np.exp(-r*(T-t))*average
barrier_uai_level=95 #Barrier of $95 for European up-and-in-call option
strike_uai=95 #Strike price of $95 for European up-and-in-call option
#All other parameters are the same as previously
european_uai_heston=np.round(uai_barrier_european_call(S_heston2,strike_uai,r,T,t,barrier_uai_level),2)
print(f"European up-and-in-call option price under Heston model with -0.70 correlation is: ${european_uai_heston}")

European up-and-in-call option price under Heston model with -0.70 correlation is: $0.26


### Q15 - Pricing European down-and-in put option under Merton model with jump intensity 0.75

In [ ]:
def dai_barrier_european_put(S,K,r,T,t,B):
    """Calculate price of down-and-in European put option using MC methods
    INPUTS: S - simulated stock paths, K - strike price,
    r - risk-free rate, T - time to maturity, t - time to calculate option price at,
    B - Barrier level for the option
    OUTPUT: Price of down-and-in option"""
    barrier_breached=(S<=B).any(axis=0)
    payoff=np.where(barrier_breached,np.maximum(K-S[-1,:],0),0)
    average=np.mean(payoff)
    return np.exp(-r*(T-t))*average
barrier_dai_level=65 #Barrier of $64 for European down-and-in put option
strike_dai=65 #Strike price of $65 for European down-and-in put option
european_dai_merton=np.round(dai_barrier_european_put(S_merton,strike_dai,r,T,t,barrier_dai_level),2)
print(f"European down-and-in put option price under Merton model with jump intensity 0.75 is: ${european_dai_merton}")

European down-and-in put option price under Merton model with jump intensity 0.75 is: $2.92
